## Data enhancement

In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

### Pre-processing images in data
** note that only training data will undergo enhancement

#### Image enhancement
The process of improving the appearance of images to highlight specific features, reduce noise or improve the quality of the image, this helps it to be more suitable for analysis.

Alpha focuses on the contrast of image
- alpha greater than 1: images brighter, enhanced contrast
- alpha less than 1: images darker, reduced contrast

Beta focuses on brightness of image
- beta positive: makes images brighter
- beta negative: makes images darker

In [2]:
train_data = pd.read_csv("datasets/csv/train_data.csv")
test_data = pd.read_csv("datasets/csv/test_data.csv")

## PREPROCESSING IMAGES WITH ENHANCEMENT
def preprocess_image(img_path, enhance):
    # reading images
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if enhance == True:
        # enhancing image
        img = cv2.convertScaleAbs(img, alpha = 1.5, beta = -20)
    # target_size of 224, 224 commonly used for image classification
    img = cv2.resize(img, (224, 224))
    # normalising pixel values
    img_array = img.astype(np.float32) / 255
    return img_array

In [3]:
# using preprocessed images as train data
train_images = np.array([preprocess_image(image_path, True) for image_path in train_data["image_path"]])
# using "pathology" column as train labels
train_labels = np.array(train_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
train_labels[train_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

In [4]:
# using preprocessed images as test data
test_images = np.array([preprocess_image(image_path, False) for image_path in test_data["image_path"]])
# using "pathology" column as test labels
test_labels = np.array(test_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
test_labels[test_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

#### Image augmentation
The augmented image stores the following:
- original image without enhancement
- enhanced image with enhancement
- all combinations of augmented flips (with enhancement)
- all combinations of augmented flips (without enhancement)

In [5]:
## IMAGE AUGMENTATION - using numpy
augmented_images = []
augmented_labels = []

# augmenting images and storing in lists
for i, img_path in enumerate(train_data["image_path"]):
    original_image = preprocess_image(img_path, False)
    enhanced_image = preprocess_image(img_path, True)
    
    # making all combinations of flips
    for horizontal_flip in [True, False]:
        for vertical_flip in [True, False]:
            # applying flips on original image
            augmented_image = original_image
            if horizontal_flip:
                augmented_image = np.fliplr(augmented_image)
            if vertical_flip:
                augmented_image = np.flipud(augmented_image)
                
            # adding augmented image and label
            augmented_images.append(augmented_image)
            augmented_labels.append(train_labels[i])
            
            # applying flips on enhanced image
            augmented_image = enhanced_image
            if horizontal_flip:
                augmented_image = np.fliplr(augmented_image)
            if vertical_flip:
                augmented_image = np.flipud(augmented_image)
                
            # adding augmented image and label
            augmented_images.append(augmented_image)
            augmented_labels.append(train_labels[i])
        
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

print(len(train_labels) * 8)
print(len(augmented_labels))

18816
18816


In [6]:
# encoding labels
label_encoder = LabelEncoder()
encoded_train_labels = label_encoder.fit_transform(train_labels)
encoded_aug_train_labels = label_encoder.fit_transform(augmented_labels)
encoded_test_labels = label_encoder.fit_transform(test_labels)
# one-hot encode labels
one_hot_train_labels = tf.keras.utils.to_categorical(encoded_train_labels)
one_hot_aug_train_labels = tf.keras.utils.to_categorical(encoded_aug_train_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(encoded_test_labels)

In [7]:
## ## VERSION 2 OF IMAGE AUGMENTATION - using TensorFlow
## datagen = ImageDataGenerator(
##     horizontal_flip = True,
##     vertical_flip = True,
##     fill_mode = "nearest"
## )
## 
## augmented_generator = datagen.flow(train_images[:, :, :, np.newaxis], one_hot_train_labels, batch_size = 32)

## Creating of models
- **Conv2D** performs convolutional operations on the input image data. It applies a set of filters to the input images to extract features.
- **MaxPooling2D** is a down-sampling operation that reduces the spatial dimensions, used after Conv2D layers to retain the most important information.
- **Flatten** is used to convert the multi-dimensional output of the previous laters into 1D.
- **Dense** represents a fully connected layer, where each neuron or node is connected to every neuron in the previous layer.

### Base model
Ensures that the data can be trained on.

In [8]:
base_model = Sequential()
# creating stack of Conv2D and MaxPooling2D
base_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
base_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
base_model.add(Flatten())
base_model.add(Dense(128, activation = "relu"))
# output layer with softmax
base_model.add(Dense(2, activation = "softmax"))

It was noted that whilst the adam optimizer had the highest accuracy, the nadam optimizer has a higher validation accuracy, which would suggest a change in choice to the preliminary report.

In [9]:
# compile model, improving accuracy
base_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = base_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

Epoch 1/10
588/588 [==============================] - 340s 574ms/step - loss: 0.7152 - accuracy: 0.5622 - val_loss: 0.6501 - val_accuracy: 0.5425
Epoch 2/10
588/588 [==============================] - 335s 570ms/step - loss: 0.6356 - accuracy: 0.6129 - val_loss: 0.6775 - val_accuracy: 0.6020
Epoch 3/10
588/588 [==============================] - 335s 570ms/step - loss: 0.6042 - accuracy: 0.6399 - val_loss: 0.6590 - val_accuracy: 0.6156
Epoch 4/10
588/588 [==============================] - 339s 576ms/step - loss: 0.5551 - accuracy: 0.6814 - val_loss: 0.6525 - val_accuracy: 0.6224
Epoch 5/10
588/588 [==============================] - 339s 576ms/step - loss: 0.5064 - accuracy: 0.7197 - val_loss: 0.7573 - val_accuracy: 0.6003
Epoch 6/10
588/588 [==============================] - 338s 574ms/step - loss: 0.4726 - accuracy: 0.7453 - val_loss: 0.7870 - val_accuracy: 0.6105
Epoch 7/10
588/588 [==============================] - 337s 574ms/step - loss: 0.4469 - accuracy: 0.7661 - val_loss: 0.7687 -

### Improving with additional layers

In [10]:
twoLayer_model = Sequential()

# first convolutional layer
twoLayer_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
twoLayer_model.add(MaxPooling2D((2, 2)))

# second convolutional layer
twoLayer_model.add(Conv2D(64, (3, 3), activation = "relu"))
twoLayer_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
twoLayer_model.add(Flatten())
twoLayer_model.add(Dense(128, activation = "relu"))
# using dropout for regularisation (reduces overfitting)
twoLayer_model.add(Dropout(0.5))
# output layer with softmax
twoLayer_model.add(Dense(2, activation = "softmax"))

In [11]:
# compile model, improving accuracy
twoLayer_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = twoLayer_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

Epoch 1/10
588/588 [==============================] - 560s 949ms/step - loss: 0.6667 - accuracy: 0.5936 - val_loss: 0.6408 - val_accuracy: 0.6020
Epoch 2/10
588/588 [==============================] - 668s 1s/step - loss: 0.5646 - accuracy: 0.6898 - val_loss: 0.6442 - val_accuracy: 0.6497
Epoch 3/10
588/588 [==============================] - 669s 1s/step - loss: 0.4201 - accuracy: 0.7912 - val_loss: 0.7147 - val_accuracy: 0.6463
Epoch 4/10
588/588 [==============================] - 670s 1s/step - loss: 0.2899 - accuracy: 0.8685 - val_loss: 0.8466 - val_accuracy: 0.6514
Epoch 5/10
588/588 [==============================] - 670s 1s/step - loss: 0.2023 - accuracy: 0.9182 - val_loss: 0.9618 - val_accuracy: 0.6412
Epoch 6/10
588/588 [==============================] - 672s 1s/step - loss: 0.1505 - accuracy: 0.9455 - val_loss: 1.1651 - val_accuracy: 0.6173
Epoch 7/10
588/588 [==============================] - 672s 1s/step - loss: 0.1258 - accuracy: 0.9538 - val_loss: 1.2172 - val_accuracy: 0.6

In [12]:
threeLayer_model = Sequential()

# first convolutional layer
threeLayer_model.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
threeLayer_model.add(MaxPooling2D((2, 2)))

# second convolutional layer
threeLayer_model.add(Conv2D(64, (3, 3), activation = "relu"))
threeLayer_model.add(MaxPooling2D((2, 2)))

# third convolutional layer
threeLayer_model.add(Conv2D(128, (3, 3), activation = "relu"))
threeLayer_model.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
threeLayer_model.add(Flatten())
threeLayer_model.add(Dense(128, activation = "relu"))
# using dropout for regularisation (reduces overfitting)
threeLayer_model.add(Dropout(0.5))
# output layer with softmax
threeLayer_model.add(Dense(2, activation = "softmax"))

In [13]:
# compile model, improving accuracy
threeLayer_model.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = threeLayer_model.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

Epoch 1/10
588/588 [==============================] - 840s 1s/step - loss: 0.6575 - accuracy: 0.5903 - val_loss: 0.6392 - val_accuracy: 0.6037
Epoch 2/10
588/588 [==============================] - 837s 1s/step - loss: 0.5863 - accuracy: 0.6653 - val_loss: 0.6382 - val_accuracy: 0.5952
Epoch 3/10
588/588 [==============================] - 844s 1s/step - loss: 0.4748 - accuracy: 0.7582 - val_loss: 0.7374 - val_accuracy: 0.6276
Epoch 4/10
588/588 [==============================] - 29545s 50s/step - loss: 0.3396 - accuracy: 0.8432 - val_loss: 0.8523 - val_accuracy: 0.6497
Epoch 5/10
588/588 [==============================] - 587s 998ms/step - loss: 0.2328 - accuracy: 0.9051 - val_loss: 1.0398 - val_accuracy: 0.6224
Epoch 6/10
588/588 [==============================] - 600s 1s/step - loss: 0.1755 - accuracy: 0.9347 - val_loss: 1.1613 - val_accuracy: 0.6139
Epoch 7/10
588/588 [==============================] - 591s 1s/step - loss: 0.1277 - accuracy: 0.9562 - val_loss: 1.2701 - val_accuracy: 